In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import sys
sys.path.append('../')
sys.argv=[''] 
del sys

In [31]:
cd "/root/SymmNeRF-baseline/code"

/root/SymmNeRF-baseline/code


In [32]:
from opt import config_parser
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import setproctitle
import tqdm 
parser = config_parser()
args = parser.parse_args()

device = torch.device("cpu")
det = args.det
lindisp = args.lindisp
args.distributed = False
args.local_feature_ch = 512
args.local_rank = -1
args.eval_scene = 'cars'
args.eval_dataset = 'srns_dataset'

args.no_load_opt = True
args.no_load_scheduler = True

setproctitle.setproctitle('[Gaetan - SymmNeRF Test]')

In [33]:
from model.feature_network import ResNetVision,ResNetSymmLocal,BasicBlock
from model.feature_network import _resnet_symm_local 

latent_dim = 256

model = _resnet_symm_local('resnet34', 
                                    latent_dim, 
                                    BasicBlock, [3, 4, 6, 3], 
                                    pretrained=False, 
                                    progress=True,
                                    use_first_layer_as_F=False).to(device)


In [34]:
x_in = torch.zeros([1,3,128,128]).to(device)

z = model(x_in)


# Check the shape of the latent code. 
print('-'*50)
print(f'Shape of latent: {z.shape}')
print('-'*50)


# Check the shape of the Feature map. 
print('-'*50)
print(f'Shape of the feature map F: {model.latent.shape}')

--------------------------------------------------
Shape of latent: torch.Size([1, 256])
--------------------------------------------------
--------------------------------------------------
Shape of the feature map F: torch.Size([1, 512, 64, 64])


#### Investigate how the latent is built in the baseline. 

In [35]:
x = model.conv1(x_in)
print(f'Shape of x after the first conv: {x.shape}')
x = model.bn1(x)
f0 = model.relu(x) # --> This first feature is saved for the futur 
x = model.maxpool(f0)
print(f'Shape of x after the maxpool: {x.shape}')

f1 = model.layer1(x) # --> Also saved. 
print(f'Shape of x after the layer1: {f1.shape}')
f2 = model.layer2(f1) # --> Also saved. 
print(f'Shape of x after the layer2: {f2.shape}')
f3 = model.layer3(f2) # --> Also saved. 
print(f'Shape of x after the layer3: {f3.shape}')

Shape of x after the first conv: torch.Size([1, 64, 64, 64])
Shape of x after the maxpool: torch.Size([1, 64, 32, 32])
Shape of x after the layer1: torch.Size([1, 64, 32, 32])
Shape of x after the layer2: torch.Size([1, 128, 16, 16])
Shape of x after the layer3: torch.Size([1, 256, 8, 8])


In [39]:
f = [f0,f1,f2,f3]
f = [feat.to('cuda:0') for feat in f]

In [45]:
from model.feature_network import LatentUpsampler
featureBlock = LatentUpsampler().to('cuda:0')
F = featureBlock(f)
print(F.shape)

torch.Size([1, 512, 64, 64])


In [38]:
import torch.nn as nn
import torch 
from model.feature_network import conv1x1,tconv5x5,LatentUpsampler




In [36]:
res = aconv1(x)
print(res.shape)

torch.Size([1, 128, 16, 16])


In [25]:
max_posenc_log_scale =9
embedding_size = 64

bvals1 = 2.**np.arange(max_posenc_log_scale)
bvals2 = 2.**np.linspace(0,max_posenc_log_scale,embedding_size//3) - 1

print(bvals1)
print(len(bvals2))

[  1.   2.   4.   8.  16.  32.  64. 128. 256.]
21


In [11]:
import numpy as np 
max_posenc_log_scale =8 
embedding_size = 64


bvals = 2.**np.linspace(0,max_posenc_log_scale,embedding_size//3) - 1
bvals = np.reshape(np.eye(3)*bvals[:,None,None], [len(bvals)*3, 3])
avals = np.ones((bvals.shape[0]))
    

[  0.           0.31950791   0.74110113   1.29739671   2.03143313
   3.           4.27803164   5.96440451   8.18958684  11.12573253
  15.          20.11212657  26.85761803  35.75834736  47.50293013
  63.          83.44850629 110.4304721  146.03338944 193.01172051
 255.        ]


In [12]:
print(bvals)

[[  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.31950791   0.           0.        ]
 [  0.           0.31950791   0.        ]
 [  0.           0.           0.31950791]
 [  0.74110113   0.           0.        ]
 [  0.           0.74110113   0.        ]
 [  0.           0.           0.74110113]
 [  1.29739671   0.           0.        ]
 [  0.           1.29739671   0.        ]
 [  0.           0.           1.29739671]
 [  2.03143313   0.           0.        ]
 [  0.           2.03143313   0.        ]
 [  0.           0.           2.03143313]
 [  3.           0.           0.        ]
 [  0.           3.           0.        ]
 [  0.           0.           3.        ]
 [  4.27803164   0.           0.        ]
 [  0.           4.27803164   0.        ]
 [  0.           0.           4.27803164]
 [  5.96440451   0.           0.        ]
 [  0.           5.96440451   0.        ]
 [  0.           0.           5.96

In [13]:
bvals = 2.**np.arange(max_posenc_log_scale)
bvals = np.reshape(np.eye(3)*bvals[:,None,None], [len(bvals)*3, 3])
avals = np.ones((bvals.shape[0])) 

print(bvals)

[[  1.   0.   0.]
 [  0.   1.   0.]
 [  0.   0.   1.]
 [  2.   0.   0.]
 [  0.   2.   0.]
 [  0.   0.   2.]
 [  4.   0.   0.]
 [  0.   4.   0.]
 [  0.   0.   4.]
 [  8.   0.   0.]
 [  0.   8.   0.]
 [  0.   0.   8.]
 [ 16.   0.   0.]
 [  0.  16.   0.]
 [  0.   0.  16.]
 [ 32.   0.   0.]
 [  0.  32.   0.]
 [  0.   0.  32.]
 [ 64.   0.   0.]
 [  0.  64.   0.]
 [  0.   0.  64.]
 [128.   0.   0.]
 [  0. 128.   0.]
 [  0.   0. 128.]]
